In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# ENABLE IF USING COLAB
USE_COLAB = True

if USE_COLAB:
    import os
    from google.colab import drive
    
    FOLDERNAME = 'UMass_Amherst/NN_AMI/NN_Project'
    drive.mount('/content/drive')
    %cd /content/drive/My\ Drive/$FOLDERNAME

    if os.path.exists('./spectroconv/'):
      %cd ./spectroconv
      !pip install hub
      !pip install hub[audio]
      !pip install wandb
    else:
      print("Error, must first pull project repository from github to Drive using ColabGitCommands.ipynb")

Mounted at /content/drive
/content/drive/My Drive/UMass_Amherst/NN_AMI/NN_Project
/content/drive/My Drive/UMass_Amherst/NN_AMI/NN_Project/spectroconv
     |████████████████████████████████| 348 kB 8.2 MB/s 
     |████████████████████████████████| 6.2 MB 62.2 MB/s 
     |████████████████████████████████| 81 kB 11.5 MB/s 
     |████████████████████████████████| 132 kB 65.8 MB/s 
     |████████████████████████████████| 79 kB 9.0 MB/s 
     |████████████████████████████████| 8.7 MB 53.9 MB/s 
     |████████████████████████████████| 138 kB 68.9 MB/s 
     |████████████████████████████████| 65 kB 4.5 MB/s 
     |████████████████████████████████| 127 kB 73.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.


In [3]:
import torch
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)


Fri Apr 29 06:42:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from models import wb_experiment

NumExpr defaulting to 2 threads.


In [8]:
n_mels = 128

config = {
  'data': {
    'train': {
        #'name': 'nsynth-tiny-train'
        'name': f"nsynth-full-{n_mels}-train"
    },
    'val': {
        #'name': 'nsynth-tiny-val'
        'name': f"nsynth-full-{n_mels}-val"
    }
  },
  'project': 'spectroconv-debug',
  'entity': 'jakeval-colab',
  'group': 'cnn',
  'job_type': 'sweep',
  'model': {
      'id': wb_experiment.ModelType.CNN,
      'name': 'CNN'
  },
}

# wandb's sweep config must be exact, no additional dictionary values are allowed.
wb_sweep_config = {
    #'name': 'cnn-tiny-dataset-sweep',
    'name': 'cnn-sweep',
    'project': 'spectroconv-debug',
    'entity': 'jakeval-colab',
    'method': 'random',
    'metric': {
      'name': 'f1',
      'goal': 'maximize'
    },
    'parameters': {
      'final_filter_size': {
          'values': [20, 40]
      },
      'num_conv_layers': {
          'values': [1,2]
      },
      'linear_layer_size': {
          'values': [48, 96]
      },
      'optimizer': {
          'value': 'adam'
      },
      'l2_reg': {
          'distribution': 'log_uniform_values',
          'min': 0.0000001,
          'max': 0.01,
      },
      'learning_rate': {
          'distribution': 'log_uniform_values',
          'min': 0.00005,
          'max': 0.01,
      },
      'dropout': {# probability of an element to be zeroed (0 = no dropout)
          'values': [0.4, 0.5]
      },
      'epochs': {
          'value': 3
      },
      'batch_size': {
          'value': 100
          #'distribution': 'q_log_uniform_values',
          #'q': 4,
          #'min': 72,
          #'max': 128,
      },
      'kernel_size_start': {
          'values': [6, 4]
      },
      'linear_reg': {
          'value': False
      }
    }
}

config['sweep_config'] = wb_sweep_config



In [ ]:
sweeper = wb_experiment.SweepExperiment(wb_config=config, login_key=True)

sweeper.run_sweep(iterations = 40, save_all = False)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: 7mhmhoze
Sweep URL: https://wandb.ai/jakeval-colab/spectroconv-debug/sweeps/7mhmhoze


wandb: Agent Starting Run: xldbtrh1 with config:
wandb: 	batch_size: 100
wandb: 	dropout: 0.5
wandb: 	epochs: 3
wandb: 	final_filter_size: 40
wandb: 	kernel_size_start: 4
wandb: 	l2_reg: 2.682401556189129e-06
wandb: 	learning_rate: 0.0004556381142100787
wandb: 	linear_layer_size: 48
wandb: 	linear_reg: False
wandb: 	num_conv_layers: 1
wandb: 	optimizer: adam


hub://jakeval/nsynth-full-128-train-metadata loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/jakeval/nsynth-full-128-train-metadata
hub://jakeval/nsynth-full-128-train loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/jakeval/nsynth-full-128-train
hub://jakeval/nsynth-full-128-val-metadata loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/jakeval/nsynth-full-128-val-metadata
hub://jakeval/nsynth-full-128-val loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/jakeval/nsynth-full-128-val


  0%|          | 0/3 [00:00<?, ?it/s]

...

hub://jakeval/nsynth-full-128-train-metadata loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/jakeval/nsynth-full-128-train-metadata
hub://jakeval/nsynth-full-128-train loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/jakeval/nsynth-full-128-train
hub://jakeval/nsynth-full-128-val-metadata loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/jakeval/nsynth-full-128-val-metadata
hub://jakeval/nsynth-full-128-val loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/jakeval/nsynth-full-128-val


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1452 [00:00<?, ?it/s]

  0%|          | 0/1452 [00:00<?, ?it/s]

  0%|          | 0/1452 [00:00<?, ?it/s]